In [1]:
import pandas as pd
import dill

## Import DEW 2019 Aqueous Species

Previously, a coder phase was generated for each species. This added a lot of bloat to the species objects- lots of derivative functions were created when only the Gibbs free energy function is required.

Here I create a new DEW specific coder phase which will produce code that will generate only the Gibbs free energy (others will generate 0).

In [2]:
from thermoengine.coder import *

In [ ]:

class DEWSpeciesModel:
    """
    Class creates representation of a DEW Species.

    Parameters
    ----------
    model_type : str
        Model type of 'TP' implies that expressions are of the form of the Gibbs 
        free energy.
        
        Model type of 'TV' implies that expressions are of the form of the 
        Helmholtz energy. This option is infrequently used.

    Attributes
    ----------
    a_list  
    expression_parts  
    g_list  
    implicit_functions  
    module  
    params  
    printer  
    variables
    
    Methods
    -------
    add_expression_to_model
    create_calc_h_file 
    create_calib_h_file
    create_code_module 
    get_berman_std_state_database
    get_include_born_code
    get_include_debye_code
    get_model_param_names
    get_model_param_symbols
    get_model_param_units
    get_module_name
    get_symbol_for_p
    get_symbol_for_pr
    get_symbol_for_t
    get_symbol_for_tr
    get_symbol_for_v
    get_symbol_for_vr
    parse_formula
    set_include_born_code
    set_include_debye_code
    set_module_name
    set_reference_origin

    Notes
    -----
    Class for creating a representation of the standard state properties of a 
    stoichiometric phase.  The principal use of the class is to construct a 
    model symbolically and to generate code that implements the model for model 
    parameter calibration and thermodynamic property calculation.

    The class supports standard state models that involve integrals of the 
    Debye function, implicit functions for equations of state, and models that
    require Born functions for estimation of the dielectric properties of 
    water.

    """
    def __init__(self, model_type='TP'):
        function_d = {"Debye":"Debye", "B":"born_B", "Y":"born_Y", "Q":"born_Q",
        "X":"born_X", "U":"born_U", "N":"born_N", "dXdT":"born_dXdT",
        "dUdT":"born_dUdT", "dNdT":"born_dNdT", "dUdP":"born_dUdP",
        "dNdP":"born_dNdP", "gSolvent":"gSolvent"}
        self._printer = SubCodePrinter(settings={"user_functions":function_d})
        self._g_list = [('g',0,0), ('dgdt',1,0), ('dgdp',0,1), ('d2gdt2',2,0), 
                        ('d2gdtdp',1,1), ('d2gdp2',0,2), ('d3gdt3',3,0), 
                        ('d3gdt2dp',2,1), ('d3gdtdp2',1,2), ('d3gdp3',0,3)]
        self._a_list = [('a',0,0), ('dadt',1,0), ('dadv',0,1), ('d2adt2',2,0), 
                        ('d2adtdv',1,1), ('d2adv2',0,2), ('d3adt3',3,0), 
                        ('d3adt2dv',2,1), ('d3adtdv2',1,2), ('d3adv3',0,3)]
        self._module = 'untitled'
        self._params = []
        self._variables = []
        self._expression_parts = []
        self._implicit_functions = []
        self._model_type = model_type
        self._T_r = 298.15 # Kelvins
        self._P_r = 1.0    # bars
        self._V_r = 2.5    # J/bar
        assert (model_type in ['TP', 'TV'])
        if model_type == 'TP':
            self._params.append(Parameter('T_r', 'K', sym.symbols('T_r')))
            self._params.append(Parameter('P_r', 'bar', sym.symbols('P_r')))
            self._variables.append(Parameter('T', 'K', sym.symbols('T')))
            self._variables.append(Parameter('P', 'bar', sym.symbols('P')))
        elif model_type == 'TV':
            self._params.append(Parameter('T_r', 'K', sym.symbols('T_r')))
            self._params.append(Parameter('V_r', 'J/bar', sym.symbols('V_r')))
            self._variables.append(Parameter('T', 'K', sym.symbols('T')))
            self._variables.append(Parameter('V', 'J/bar', sym.symbols('V')))
        else:
            print ('ERROR: Unsupported model_type. Must be either "TP" or "TV"')
        self._elements = pd.read_csv(path.join(path.dirname(__file__), DATADIR, 
            'elements.csv'))
        self._entropies = pd.read_csv(path.join(path.dirname(__file__), DATADIR, 
            'entropies.csv'))
        self._berman_db = None
        self._include_debye_code = False
        self._include_born_code = False
        self.calc_h = None
        self.calib_h = None
        self.fast_h_template = None
        self.calib_h_template = None
        self.fast_c_template = None 
        self.calib_c_template = None
        return

    @property
    def a_list(self):
        """
        Array of tuples used internally by the class
        
        Each tuple contains the following entries:   
    
        - [0] str - Function name for derivatives of the Helmholtz energy         
        - [1] int - Order of temperature derivative      
        - [2] int - Order of volume derivative
          
        Returns
        -------
        Array of tuples
        """
        return self._a_list
    
    @a_list.setter
    def a_list(self, a_list):
        self._a_list = a_list

    @property
    def g_list(self):
        """
        Array of tuples used internally by the class
        
        Each tuple contains the following entries:

        - [0] str - Function name for derivatives of the Gibbs free energy      
        - [1] int - Order of temperature derivative     
        - [2] int - Order of pressure derivative
        
        Generally for internal use by the class

        Returns
        -------
        Array of tuples
        """     
        return self._g_list
    
    @g_list.setter
    def g_list(self, g_list):
        self._g_list = g_list

    @property
    def expression_parts(self):
        """     
        Array of Expression class instances  

        A model is built by layering expression instances. Each instance may
        be applicable over all or a portion of the domain space of variables.       

        Returns
        -------
        Array of Expression class instances, [str,...]
        """ 
        return self._expression_parts
    
    @expression_parts.setter
    def expression_parts(self, expression_parts):
        self._expression_parts = expression_parts

    @property
    def implicit_functions(self):
        """     
        Array of Implicit_Function class instances

        A model may require that one or more implicit function expressions be 
        satisfied prior to evaluation of the model expressions. 
   
        Returns
        -------
        Array of Implicit_Function class instances, [str,...]
        """ 
        return self._implicit_functions
    
    @implicit_functions.setter
    def implicit_functions(self, implicit_functions):
        self._implicit_functions = implicit_functions

    @property
    def module(self):
        """     
        Name of module
                
        Returns
        -------
        Name of module (str)
        """
        return self._module

    @module.setter
    def module(self, module):
        assert isinstance(module, str)
        self._module = module

    @property
    def params(self):
        """     
        Array of Parameter class instances  

        Parameters are calibrated quantities that define the model, like T_r 
        and P_r.
        
        Returns
        -------
        Array of Parameter class instances, [Parameter,...]
        """
        return self._params
    
    @params.setter
    def params(self, params):
        self._params = params

    @property
    def printer(self):
        """
        Instance of a SymPy code printer class used in code generation
        
        Returns
        -------
        C99CodePrinter
        """
        return self._printer
    
    @printer.setter
    def printer(self, printer):
        self._printer = printer

    @property
    def variables(self):
        """
        Array of Parameter class instances  
        
        This property returns or sets an array of variables, either T, P, T_r and P_r 
        (if model_type is ’TP’) or T, V, T_r and V_r (if model type is equal to ’TV’). 
        Array elements are encapsulated as instances of the Parameter class object. 
        This is simply done as a convenience wrapper. Note that SymPy symbols that are 
        suitable for developing model expressions using these variables are probably 
        more easily accessed using the methods:
        
        - get_symbol_for_t()    
        - get_symbol_for_p()
        
        and so on.  
                
        Returns
        -------
        Array of Parameter class instances, [Parameter,...]
        """
        return self._variables
    
    @variables.setter
    def variables(self, variables):
        self._variables = variables

    def get_symbol_for_t(self):
        """
        Retrieves SymPy symbol for temperature.

        Returns
        -------
        T : sympy.core.symbol.Symbol
            SymPy symbol for temperature
        """
        for x in self.variables:
            if x.name == 'T':
                return x.expression
        return None

    def get_symbol_for_p(self):
        """
        Retrieves SymPy symbol for pressure.

        Returns
        -------
        P : sympy symbol
            SymPy symbol for pressure
        """
        for x in self.variables:
            if x.name == 'P':
                return x.expression
        return None

    def get_symbol_for_v(self):
        """
        Retrieves SymPy symbol for volume.

        Returns
        -------
        V : sympy symbol
            SymPy symbol for volume
        """
        for x in self.variables:
            if x.name == 'V':
                return x.expression
        return None

    def get_symbol_for_tr(self):
        """
        Retrieves SymPy symbol for reference temperature.

        Returns
        -------
        Tr : sympy symbol
             SymPy symbol for reference temperature
        """
        for x in self.params:
            if x.name == 'T_r':
                return x.expression
        return None

    def get_symbol_for_pr(self):
        """
        Retrieves SymPy symbol for reference pressure.

        Returns
        -------
        Pr : sympy symbol
             SymPy symbol for reference pressure
        """
        for x in self.params:
            if x.name == 'P_r':
                return x.expression
        return None

    def get_symbol_for_vr(self):
        """
        Retrieves SymPy symbol for reference volume.

        Returns
        -------
        Vr : sympy symbol
             SymPy symbol for reference volume
        """
        for x in self.params:
            if x.name == 'V_r':
                return x.expression
        return None

    def get_module_name(self):
        """
        Retrieves module name.

        The module name is used in creating file names for coded functions.
        Maintained for backward compatibility.

        Returns
        -------
        string : str
            The name of the module.
        """
        return self._module

    def set_module_name(self, module_s):
        """
        Retrieves module name.

        The module name is used in creating file names for coded functions.
        Maintained for backward compatibility.

        Parameters
        ----------
        module_s : str
            The name of the module
        """
        self._module = module_s

    def set_reference_origin(self, Tr=298.15, Pr=1.0, Vr=2.5):
        """
        Sets the reference conditions for the model.

        Tr must be specified along with one of Pr or Vr.

        Parameters
        ----------
        Tr : float
             Reference temperature for the model (in Kelvins)
        Pr : float
             Reference pressure for the model (in bars)
        Vr : float
             Reference volume of the model (in J/bar)
        """
        self._T_r = Tr
        self._P_r = Pr
        self._V_r = Vr

    def get_include_debye_code(self):
        """
        Retrieves a boolean flag specifying whether a block of code 
        implementing the Debye integral function will be generated.

        Returns
        -------
        boolean : bool
                  True or False (default)
        """
        return self._include_debye_code

    def set_include_debye_code(self,include=False):
        """
        Sets a boolean flag controlling the inclusion of a block of code 
        implementing the Debye integral function.

        Parameters
        ----------
        include : bool
                  True or False
        """
        if include:
            self._include_debye_code = True
        else:
            self._include_debye_code = False

    def get_include_born_code(self):
        """
        Retrieves a boolean flag specifying whether code implementing the Born 
        functions will be linked to the generated module.

        Returns
        -------
        boolean : bool
                  True or False (default)
        """
        return self._include_born_code

    def set_include_born_code(self,include=False):
        """
        Sets a boolean flag controlling the inclusion of code implementing the 
        Born functions.

        Parameters
        ----------
        include : bool
                  True or False
        """
        if include:
            self._include_born_code = True
        else:
            self._include_born_code = False

    def get_model_param_names(self):
        """
        Retrieves a list of strings of model parameter names.

        Returns
        -------
        result : list of str
            Names of model parameters
        """
        result = []
        for x in self.params:
            result.append(x.name)
        return result

    def get_model_param_units(self):
        """
        Retrieves a list of strings of model parameter units.

        Returns
        -------
        result : list of str
            Units of model parameters
        """
        result = []
        for x in self.params:
            result.append(x.units)
        return result

    def get_model_param_symbols(self):
        """
        Retrieves a list of SymPy symbols for model parameters.

        Returns
        -------
        result : list of sym.Symbol
            SymPy symbols for model parameters
        """
        result = []
        for x in self.params:
            result.append(x.expression)
        return result

    def add_expression_to_model(self, expression, params, 
        exp_type='unrestricted',
        lower_limits=(None, None), upper_limits=(None, None), 
        implicit_functions=None, extend_restricted_functions=True):
        """
        Adds an expression and associated parameters to the model.

        Adds an expression for the Gibbs or Helmholtz energy to the 
        standard state model along with a description of expression parameters.

        Parameters
        ----------
        expression : sympy.core.symbol.Symbol
            A SymPy expression for the Gibbs free energy (if model_type is 'TP')
            or the Helmholtz energy (if model_type is 'TV').
            
            The expression may contain an implicit variable, *f*, whose value
            is a function of T,P or T,V.  The value of *f* is determined 
            numerically by solving the implicit_function expression (defined 
            below) at runtime using Newton's method.

        params : An array of tuples
            Structure (string, string, SymPy expression):
            
            - [0] str - Name of the parameter
            
            - [1] str - Units of parameter
            
            - [2] sympy.core.symbol.Symbol - SymPy symbol for the parameter

        exp_type : str, default='unrestricted'
            - 'unrestricted' - Expression is applicable over the whole of T,P or T,V space.
            
            - 'restricted' - Expression applies only between the specified 
              lower_limits and upper_limits.

        lower_limits : tuple
            A tuple of SymPy expressions defining the inclusive lower (T,P) or 
            (T,V) limit of applicability of the expression. Used only if 
            exp_type is set to 'restricted'.

        upper_limits : tuple
            A tuple of SymPy expressions defining the inclusive upper (T,P) or 
            (T,V) limit of applicability of the expression. Used only if 
            exp_type is set to 'restricted'.

        implicit_functions : array of tuples
            A tuple element contains three parts:
             
            - [0] sympy.core.symbol.Symbol - SymPy expression for an implicit 
              function in two independent (T,P or T,V) variables and one 
              dependent variable (*f*). The function is equal to zero.     
            
            - [1] sympy.core.symbol.Symbol - SymPy symbol for the dependent 
              variable *f*. 
            
            - [2] sympy.core.symbol.Symbol - SymPy expression that initializes *f* 
              in the iterative routine. This expression must be defined in 
              terms of known parameters and Tr, Pr, T, P.

        extend_restricted_functions : bool, default=True
            A boolean that controls whether a "restricted" *exp_type* is 
            extended beyond its *upper_limits*. By default, temperature and 
            pressure/volume derivatives of *expression* are evaluated at the
            *upper_limits*, and these constants are added as linear functions of
            *T* and *P/V* applicable and restricted to conditions *above* the 
            *upper_limits*. You can disable the default behavior by setting 
            *extend_restricted_functions* to *False*.  Note that, in general, the
            default behavior is desired as it allows entropic and volumetric 
            contributions developed over the restricted domain of *expression* 
            to contribute to the energy potential beyond the domain. Such 
            behavior is consistent with SymPy Piecewise functions. Special 
            circumstances, however, may require the default behavior to be
            disabled. 
        """
        l_params = []
        for (x,y,z) in params:
            l_params.append(Parameter(x,y,z))
        for x in l_params:
            self._params.append(x)

        l_exp = Expression(expression, l_params, 
            exp_type=exp_type, 
            lower_limit_T=lower_limits[0], lower_limit_PorV=lower_limits[1],
            upper_limit_T=upper_limits[0], upper_limit_PorV=upper_limits[1])
        self._expression_parts.append(l_exp)

        if implicit_functions:
            for (x,y,z) in implicit_functions:
                self._implicit_functions.append(Implicit_Function(x,y,z))

        if not extend_restricted_functions:
            return 

        if upper_limits[0] == None and upper_limits[1] == None:
            return

        elif upper_limits[0] != None and upper_limits[1] != None:
            if self._model_type == 'TP':
                T = self.get_symbol_for_t()
                P = self.get_symbol_for_p()
                s = -expression.diff(T).subs(T,upper_limits[0]).subs(
                    P,upper_limits[1])
                v =  expression.diff(P).subs(T,upper_limits[0]).subs(
                    P,upper_limits[1])
                g =  -(T-upper_limits[0])*s + (P-upper_limits[1])*v
                self.expression_parts.append(
                    Expression(g, l_params, exp_type='restricted',
                        lower_limit_T=upper_limits[0], 
                        lower_limit_PorV=upper_limits[1],
                        upper_limit_T=None,
                        upper_limit_PorV=None)
                    )
            elif self._model_type == 'TV':
                T = self.get_symbol_for_t()
                V = self.get_symbol_for_v()
                s = -expression.diff(T).subs(T,upper_limits[0]).subs(
                    V,upper_limits[1])
                p = -expression.diff(V).subs(T,upper_limits[0]).subs(
                    V,upper_limits[1])
                a =  -(T-upper_limits[0])*s - (V-upper_limits[1])*p
                self.expression_parts.append(
                    Expression(a, l_params, exp_type='restricted', 
                        lower_limit_T=upper_limits[0], 
                        lower_limit_PorV=upper_limits[1], 
                        upper_limit_T=None, 
                        upper_limit_PorV=None)
                    )
        elif upper_limits[0] != None:
            if self._model_type == 'TP':
                T = self.get_symbol_for_t()
                s = -expression.diff(T).subs(T,upper_limits[0])
                g =  -(T-upper_limits[0])*s
                self.expression_parts.append(
                    Expression(g, l_params, exp_type='restricted', 
                        lower_limit_T=upper_limits[0], 
                        lower_limit_PorV=upper_limits[1], 
                        upper_limit_T=None, 
                        upper_limit_PorV=None)
                    )
            elif self._model_type == 'TV':
                T = self.get_symbol_for_t()
                s = -expression.diff(T).subs(T,upper_limits[0])
                a =  -(T-upper_limits[0])*s
                self.expression_parts.append(
                    Expression(a, l_params, exp_type='restricted', 
                        lower_limit_T=upper_limits[0], 
                        lower_limit_PorV=upper_limits[1], 
                        upper_limit_T=None, 
                        upper_limit_PorV=None)
                    )
        elif upper_limits[1] != None:
            if self._model_type == 'TP':
                P = self.get_symbol_for_p()
                v =  expression.diff(P).subs(P,upper_limits[1])
                g =  (P-upper_limits[1])*v
                self.expression_parts.append(
                    Expression(g, l_params, exp_type='restricted', 
                        lower_limit_T=upper_limits[0], 
                        lower_limit_PorV=upper_limits[1], 
                        upper_limit_T=None, 
                        upper_limit_PorV=None)
                    )
            elif self._model_type == 'TV':
                V = self.get_symbol_for_v()
                p = -expression.diff(V).subs(V,upper_limits[1])
                a = -(V-upper_limits[1])*p
                self.expression_parts.append(
                    Expression(a, l_params, exp_type='restricted', 
                        lower_limit_T=upper_limits[0], 
                        lower_limit_PorV=upper_limits[1], 
                        upper_limit_T=None, 
                        upper_limit_PorV=None)
                    )

    def create_calc_h_file(self, language='C', module_type='fast'):
        """
        Creates an include file implementing model calculations.

        Note that this include file contains code for functions that implement 
        the model for the generic case. It is meant to be included into a file 
        that implements a specific parameterized instance of the model. See 
        create_fast_code_module().

        The user does not normally call this function directly.

        Parameters
        ----------
        language : str
            Language syntax for generated code. ("C" is the C99 programming 
            language.)
        module_type : str
            Generate code that executes "fast" but does not expose hooks for 
            model parameter calibration. Alternately, generate code suitable for 
            "calib"ration of parameters in the model, which executes more slowly 
            and exposes additional functions that allow setting of parameters 
            and generation of derivatives of thermodynamic functions with 
            respect to model parameters. 
        """
        assert language == "C"
        s = "#include <math.h>\n\n"
        if self._include_debye_code:
            s += tpl.create_code_for_debye_function(language)
        if self._include_born_code:
            s += tpl.create_code_for_born_functions(language)
            s += "\n"
        if self._model_type == 'TP':
            T = self.get_symbol_for_t()
            P = self.get_symbol_for_p()
            implicit_calls = ""
            # list of lists [][], rows denote temperature derivatives, columns, 
            # pressure derivatives
            # each element contains a tuple: (function, symbol for function, 
            # sympy expression for function,
            # sympy expression for fully substituted function)
            self.global_subs = []
            if len(self._implicit_functions) > 0:
                for index,imp in enumerate(self._implicit_functions):
                    fn       = imp.function
                    var      = imp.variable
                    var_init = imp.guess
                    var_str  = "var" + str(index)
                    var_sym  = sym.symbols(var_str)
                    fn_subs  = fn.subs(var,var_sym)
                    if module_type == 'fast':
                        s += ("static double " + var_str + " = " 
                            + self.printer.doprint(var_init) + ";\n")
                    elif module_type == 'calib':
                        s += "static double " + var_str + " = -1.0;\n"
                    implicit_calls += ("    " + self.module + "_" + var_str 
                        + "(T, P);\n")
                    for oT in range(0,4):
                        self.global_subs.append([])
                        for oP in range(0,4):
                            if oT+oP == 0:
                                # definition of the primary implicit variable
                                self.global_subs[0].append(
                                    (var, var_str, var_str, var))
                            elif oT+oP <= 3:
                                var_diff = sym.diff(var,T,oT,P,oP)
                                fn_diff = sym.solve(sym.diff(fn,T,oT,P,oP),
                                    var_diff)[0]
                                var_sym_diff = (sym.symbols("d"+str(oT+oP)+"var"
                                    +str(index)+"d"+str(oT)+"Td"+str(oP)+"P"))
                                fn_sym_diff = fn_diff
                                ss_list = []
                                for oo in self.global_subs:
                                    for (aa,bb,cc,dd) in oo:
                                        ss_list.insert(0,(aa,cc))
                                fn_sym_diff = fn_sym_diff.subs(ss_list)
                                self.global_subs[oT].append(
                                    (var_diff, fn_diff, var_sym_diff, 
                                        fn_sym_diff))
                    s += ("static void " + self.module + "_" + var_str 
                        + "(double T, double P) {\n")
                    s += "    static double Told = 0.0;\n"
                    s += "    static double Pold = 0.0;\n"
                    if module_type == 'calib':
                        s += "    if (" + var_str + " == -1.0) {\n"
                        s += ("        " + var_str + " = " 
                            + self.printer.doprint(var_init) + ";\n")
                        s += "    }\n"
                    s += "    if ((T != Told) && (P != Pold)) {\n"
                    s += "        Told = T;\n"
                    s += "        Pold = P;\n"
                    s += "        double f = 0.0;\n"
                    s += "        int iter = 0;\n"
                    s += "        do {\n"
                    s += "            f = " + self.printer.doprint(fn_subs) + ";\n"
                    s += "            double df = " 
                    s += self.printer.doprint(sym.diff(fn_subs,var_sym)) + ";\n"
                    s += "            if (df == 0.0) break;\n"
                    s += "            " + var_str + " -= f/df;\n"
                    s += "            if (" + var_str + " <= 0.0) " 
                    s += var_str + " = 0.001;\n"
                    s += "            else if (" + var_str + " >= 2.0*V_TrPr) " 
                    s += var_str + " = 2.0*V_TrPr;\n"
                    s += "            iter++;\n"
                    s += "        } while ((fabs(f) > 0.001) && (iter < 200));\n"
                    s += "    }\n"
                    s += "}\n\n"
            for (f, oT, oP) in self.g_list:
                s += ("static double " + self.module + "_" + f 
                    + "(double T, double P) {\n")
                s += implicit_calls
                s += "    double result = 0.0;\n"
                #
                ss_list = []
                if len(self.global_subs) > 0:
                    ss_index = list(product([i for i in range(0,oT+1)],
                        [i for i in range(0,oP+1)]))
                    for (ooT,ooP) in ss_index:
                        if ooT+ooP > 0:
                            s += "    double " 
                            s += self.printer.doprint(self.global_subs[ooT][ooP][2]) 
                            s += " = "
                            s += self.printer.doprint(self.global_subs[ooT][ooP][3]) 
                            s += ";\n"
                        ss_list.insert(0,(self.global_subs[ooT][ooP][0], 
                            self.global_subs[ooT][ooP][2]))
                #
                inited = False
                for exp in self.expression_parts:
                    a = sym.diff(exp.expression,T,oT,P,oP).subs(ss_list)
                    if a == sym.S.Zero:
                        pass
                    elif exp.exp_type == 'unrestricted':
                        s += "    result += " + self.printer.doprint(a) + ";\n"
                        inited = True
                    elif exp.exp_type == 'restricted':
                        test_term = "    if("
                        separator = ""
                        for param in exp.params:
                            test_term += (separator + "((" 
                                       + self.printer.doprint(param.expression) 
                                       + ") != 0.0)")
                            separator = " || "
                        test_term += ") {\n"
                        s += test_term
                        test_term = "        if("
                        separator = ""
                        if exp.lower_limit_T != None:
                            test_term += (separator + "(T > (" 
                                + self.printer.doprint(exp.lower_limit_T) 
                                + "))")
                            separator = " && "
                        if exp.lower_limit_PorV != None:
                            test_term += (separator + "(P > (" 
                                + self.printer.doprint(exp.lower_limit_PorV) 
                                + "))")
                            separator = " && "
                        if exp.upper_limit_T != None:
                            test_term += (separator + "(T <= (" 
                                + self.printer.doprint(exp.upper_limit_T) 
                                + "))")
                            separator = " && "
                        if exp.upper_limit_PorV != None:
                            test_term += (separator + "(P <= (" 
                                + self.printer.doprint(exp.upper_limit_PorV) 
                                + "))")
                            separator = " && "
                        test_term += ") {\n"
                        s += test_term
                        s += "             result += " 
                        s += self.printer.doprint(a) + ";\n"
                        s += "        }\n"
                        s += "    }\n"
                        inited = True
                if not inited:
                    s += "    result += 0.0;\n"
                s += "    return result;\n}\n\n"
        elif self._model_type == 'TV':
            T = self.get_symbol_for_t()
            V = self.get_symbol_for_v()
            implicit_calls = ""
            # list of lists [][], rows denote temperature derivatives, columns, 
            # volume derivatives
            # each element contains a tuple: (function, symbol for function, 
            # sympy expression for function,
            # sympy expression for fully substituted function)
            self.global_subs = []
            if len(self._implicit_functions) > 0:
                for index,imp in enumerate(self._implicit_functions):
                    fn       = imp.function
                    var      = imp.variable 
                    var_init = imp.guess
                    var_str = "var" + str(index)
                    var_sym = sym.symbols(var_str)
                    fn_subs = fn.subs(var,var_sym)
                    if module_type == 'fast':
                        s += ("static double " + var_str + " = " 
                            + self.printer.doprint(var_init) + ";\n")
                    elif module_type == 'calib':
                        s += "static double " + var_str + " = -1.0;\n"
                    implicit_calls += ("    " + self.module + "_" + var_str 
                        + "(T, V);\n")
                    for oT in range(0,4):
                        self.global_subs.append([])
                        for oV in range(0,4):
                            if oT+oV == 0:
                                # definition of the primary implicit variable
                                self.global_subs[0].append((var, var_str, 
                                    var_str, var))
                            elif oT+oV <= 3:
                                var_diff = sym.diff(var,T,oT,V,oV)
                                fn_diff = sym.solve(sym.diff(fn,T,oT,V,oV),
                                    var_diff)[0]
                                var_sym_diff = sym.symbols("d"+str(oT+oV)+"var"
                                    +str(index)+"d"+str(oT)+"Td"+str(oV)+"V")
                                fn_sym_diff = fn_diff
                                ss_list = []
                                for oo in self.global_subs:
                                    for (aa,bb,cc,dd) in oo:
                                        ss_list.insert(0,(aa,cc))
                                fn_sym_diff = fn_sym_diff.subs(ss_list)
                                self.global_subs[oT].append((var_diff, fn_diff, 
                                    var_sym_diff, fn_sym_diff))
                    s += "static void " + self.module + "_" 
                    s += var_str + "(double T, double V) {\n"
                    s += "    static double Told = 0.0;\n"
                    s += "    static double Vold = 0.0;\n"
                    if module_type == 'calib':
                        s += "    if (" + var_str + " == -1.0) {\n"
                        s += "        " + var_str + " = " 
                        s += self.printer.doprint(var_init) + ";\n"
                        s += "    }\n"
                    s += "    if ((T != Told) && (V != Vold)) {\n"
                    s += "        Told = T;\n"
                    s += "        Vold = V;\n"
                    s += "        double f = 0.0;\n"
                    s += "        int iter = 0;\n"
                    s += "        do {\n"
                    s += "            f = " + self.printer.doprint(fn_subs) + ";\n"
                    s += "            double df = " 
                    s += self.printer.doprint(sym.diff(fn_subs,var_sym)) + ";\n"
                    s += "            if (df == 0.0) break;\n"
                    s += "            " + var_str + " -= f/df;\n"
                    s += "            if (" + var_str + " <= 0.0) " 
                    s += var_str + " = 0.001;\n"
                    s += "            else if (" + var_str + " >= 4.0*" 
                    s += self.printer.doprint(var_init) 
                    s +=                  +") " + var_str + " = 4.0*" 
                    s += self.printer.doprint(var_init) + ";\n"
                    s += "            iter++;\n"
                    s += "        } while ((fabs(f) > 0.001) && (iter < 200));\n"
                    s += "    }\n"
                    s += "}\n\n"
            for (f, oT, oV) in self.a_list:
                s += ("static double " + self.module + "_" + f 
                    + "(double T, double V) {\n")
                s += implicit_calls
                s += "    double result = 0.0;\n"
                #
                ss_list = []
                if len(self.global_subs) > 0:
                    ss_index = list(product([i for i in range(0,oT+1)],
                        [i for i in range(0,oV+1)]))
                    for (ooT,ooV) in ss_index:
                        if ooT+ooV > 0:
                            s += "    double " 
                            s += self.printer.doprint(self.global_subs[ooT][ooV][2]) 
                            s += " = "
                            s += self.printer.doprint(self.global_subs[ooT][ooV][3]) 
                            s += ";\n"
                        ss_list.insert(0,(self.global_subs[ooT][ooV][0], 
                            self.global_subs[ooT][ooV][2]))
                #
                inited = False
                for exp in self.expression_parts:
                    a = sym.diff(exp.expression,T,oT,V,oV).subs(ss_list)
                    if a == sym.S.Zero:
                        pass
                    elif exp.exp_type == 'unrestricted':
                        s += "    result += " + self.printer.doprint(a) + ";\n"
                        inited = True
                    elif exp.exp_type == 'restricted':
                        test_term = "    if("
                        separator = ""
                        for param in exp.params:
                            test_term += (separator + "((" 
                                + self.printer.doprint(param.expression) 
                                + ") != 0.0)")
                            separator = " || "
                        test_term += ") {\n"
                        s += test_term
                        test_term = "        if("
                        separator = ""
                        if exp.lower_limit_T != None:
                            test_term += (separator + "(T > (" 
                                + self.printer.doprint(exp.lower_limit_T) 
                                + "))")
                            separator = " && "
                        if exp.lower_limit_PorV != None:
                            test_term += (separator + "(V > (" 
                                + self.printer.doprint(exp.lower_limit_PorV) 
                                + "))")
                            separator = " && "
                        if exp.upper_limit_T != None:
                            test_term += (separator + "(T <= (" 
                                + self.printer.doprint(exp.upper_limit_T) 
                                + "))")
                            separator = " && "
                        if exp.upper_limit_PorV != None:
                            test_term += (separator + "(V <= (" 
                                + self.printer.doprint(exp.upper_limit_PorV) 
                                + "))")
                            separator = " && "
                        test_term += ") {\n"
                        s += test_term
                        s += "             result += " + self.printer.doprint(a) 
                        s += ";\n"
                        s += "        }\n"
                        s += "    }\n"
                        inited = True
                if not inited:
                    s += "    result += 0.0;\n"
                s += "    return result;\n}\n\n"
        else:
            print ("Unsupported model_type: ", self._model_type)
            s = ""
        self.calc_h = (s + tpl.create_redundant_function_template(
            model_type=self._model_type).format(module=self.module,
            v_initial_guess="2.0"))

    def create_calib_h_file(self, language='C'):
        """
        Creates a C-code include file implementing model calibration functions.

        Parameters
        ----------
        language : string
            Language syntax for generated code. ("C" is the C99 programming 
            language.)

        Notes
        -----
        The calib_h include file contains code for functions that implement 
        the model for the generic case. It is meant to be included into a file 
        that implements a specific parameterized instance of the model. 
        See create_calib_code_module().

        The user does not normally call this function directly.
        """
        assert language == "C"
        s = "#include <math.h>\n\n"
        if self._model_type == 'TP':
            T = self.get_symbol_for_t()
            P = self.get_symbol_for_p()
            for (f, oT, oP) in self.g_list:
                s += ("static double " + self.module + "_dparam_" + f 
                    + "(double T, double P, int index) {\n")
                s += "    double result = 0.0;\n"
                #
                ss_list = []
                if len(self.global_subs) > 0:
                    ss_index = list(product([i for i in range(0,oT+1)],
                        [i for i in range(0,oP+1)]))
                    for (ooT,ooP) in ss_index:
                        if ooT+ooP > 0:
                            s += "    double " 
                            s += self.printer.doprint(self.global_subs[ooT][ooP][2]) 
                            s += " = "
                            s += self.printer.doprint(self.global_subs[ooT][ooP][3]) 
                            s += ";\n"
                        ss_list.insert(0,(self.global_subs[ooT][ooP][0], 
                            self.global_subs[ooT][ooP][2]))
                #
                s += "    switch (index) {\n"
                for i in range (0,len(self.params)):
                    s += ("    case " + str(i) + ": /* " 
                        + self.printer.doprint(self.params[i].expression) 
                        + " */ \n")
                    #
                    inited = False
                    for exp in self.expression_parts:
                        a = sym.diff(exp.expression,T,oT,P,oP,
                            self.params[i].expression,1).subs(ss_list)
                        if a == sym.S.Zero:
                            pass
                        elif exp.exp_type == 'unrestricted':
                            s += "        result += " 
                            s += self.printer.doprint(a) + ";\n"
                            inited = True
                        elif exp.exp_type == 'restricted':
                            test_term = "        if("
                            separator = ""
                            for param in exp.params:
                                test_term += (separator + "((" 
                                    + self.printer.doprint(param.expression) 
                                    + ") != 0.0)")
                                separator = " || "
                            test_term += ") {\n"
                            s += test_term
                            test_term = "            if("
                            separator = ""
                            if exp.lower_limit_T != None:
                                test_term += (separator + "(T > (" 
                                    + self.printer.doprint(exp.lower_limit_T) 
                                    + "))")
                                separator = " && "
                            if exp.lower_limit_PorV != None:
                                test_term += (separator + "(P > (" 
                                    + self.printer.doprint(exp.lower_limit_PorV) 
                                    + "))")
                                separator = " && "
                            if exp.upper_limit_T != None:
                                test_term += (separator + "(T <= (" 
                                    + self.printer.doprint(exp.upper_limit_T) 
                                    + "))")
                                separator = " && "
                            if exp.upper_limit_PorV != None:
                                test_term += (separator + "(P <= (" 
                                    + self.printer.doprint(exp.upper_limit_PorV) 
                                    + "))")
                                separator = " && "
                            test_term += ") {\n"
                            s += test_term
                            s += "                 result += " 
                            s += self.printer.doprint(a) + ";\n"
                            s += "            }\n"
                            s += "        }\n"
                            inited = True
                    #
                    if not inited:
                        s += "        result += 0.0;\n"
                    s += "        break;\n"
                s += "    }\n"
                s += "    return result;\n}\n\n"
        elif self._model_type == 'TV':
            T = self.get_symbol_for_t()
            V = self.get_symbol_for_v()
            for (f, oT, oV) in self.a_list:
                s += ("static double " + self.module + "_dparam_" + f 
                    + "(double T, double V, int index) {\n")
                s += "    double result = 0.0;\n"
                #
                ss_list = []
                if len(self.global_subs) > 0:
                    ss_index = list(product([i for i in range(0,oT+1)],
                        [i for i in range(0,oV+1)]))
                    for (ooT,ooV) in ss_index:
                        if ooT+ooV > 0:
                            s += "    double " 
                            s += self.printer.doprint(self.global_subs[ooT][ooV][2])
                            s += " = "
                            s += self.printer.doprint(self.global_subs[ooT][ooV][3]) 
                            s += ";\n"
                        ss_list.insert(0,(self.global_subs[ooT][ooV][0], 
                            self.global_subs[ooT][ooV][2]))
                #
                s += "    switch (index) {\n"
                for i in range (0,len(self.params)):
                    s += "    case " + str(i) + ": /* " 
                    s += self.printer.doprint(self.params[i].expression) 
                    s += " */ \n"
                    #
                    inited = False
                    for exp in self.expression_parts:
                        a = sym.diff(exp.expression,T,oT,V,oV,
                            self.params[i].expression,1).subs(ss_list)
                        if a == sym.S.Zero:
                            pass
                        elif exp.exp_type == 'unrestricted':
                            s += "        result += " + self.printer.doprint(a) 
                            s += ";\n"
                            inited = True
                        elif exp.exp_type == 'restricted':
                            test_term = "        if("
                            separator = ""
                            for param in exp.params:
                                test_term += (separator + "((" 
                                    + self.printer.doprint(param.expression) 
                                    + ") != 0.0)")
                                separator = " || "
                            test_term += ") {\n"
                            s += test_term
                            test_term = "            if("
                            separator = ""
                            if exp.lower_limit_T != None:
                                test_term += (separator + "(T > (" 
                                    + self.printer.doprint(exp.lower_limit_T) 
                                    + "))")
                                separator = " && "
                            if exp.lower_limit_PorV != None:
                                test_term += (separator + "(V > (" 
                                    + self.printer.doprint(exp.lower_limit_PorV) 
                                    + "))")
                                separator = " && "
                            if exp.upper_limit_T != None:
                                test_term += (separator + "(T <= (" 
                                    + self.printer.doprint(exp.upper_limit_T) 
                                    + "))")
                                separator = " && "
                            if exp.upper_limit_PorV != None:
                                test_term += (separator + "(V <= (" 
                                    + self.printer.doprint(exp.upper_limit_PorV) 
                                    + "))")
                                separator = " && "
                            test_term += ") {\n"
                            s += test_term
                            s += "                 result += " 
                            s += self.printer.doprint(a) + ";\n"
                            s += "            }\n"
                            s += "        }\n"
                            inited = True
                    #
                    if not inited:
                        s += "        result += 0.0;\n"
                    s += "        break;\n"
                s += "    }\n"
                s += "    return result;\n}\n\n"
            s += tpl.create_redundant_calib_TV_template().format(
                module=self.module)
        else:
            print ("Unsupported model_type: ", self._model_type)
            s = ""
        
        # Additional functions
        s += "static int " + self.module + "_get_param_number(void) {\n"
        s += "    return " + str(len(self.params)) + ";\n"
        s += "}\n\n"
        #
        s += "static const char *paramNames["  + str(len(self.params)) + "] = {"
        separator = " "
        for param in self.params:
            s += separator + '"' + self.printer.doprint(param.expression) + '"'
            separator = ", "
        s += "  };\n\n"
        #
        s += "static const char *paramUnits["  + str(len(self.params)) + "] = {"
        separator = " "
        for param in self.params:
            s += separator + '"' + param.units + '"'
            separator = ", "
        s += "  };\n\n"
        #
        s += "static const char **" + self.module + "_get_param_names(void) {\n"
        s += "    return paramNames;\n"
        s += "}\n\n"
        #
        s += "static const char **" + self.module + "_get_param_units(void) {\n"
        s += "    return paramUnits;\n"
        s += "}\n\n"
        #
        s += "static void " + self.module + "_get_param_values(double **values) {\n"
        for i in range(0, len(self.params)):
            s += "    (*values)[" + str(i) + "] = " 
            s += self.printer.doprint(self.params[i].expression) + ";\n"
        s += "}\n\n"
        #
        s += "static int " + self.module + "_set_param_values(double *values) {\n"
        for i in range(0, len(self.params)):
            s += "    " + self.printer.doprint(self.params[i].expression) 
            s += "= values[" + str(i) + "];\n"
        s += "    return 1;\n"
        s += "}\n\n"
        #
        s += "static double " + self.module + "_get_param_value(int index) {\n"
        s += "    double result = 0.0;\n"
        s += "    switch (index) {\n"
        for i in range(0, len(self.params)):
            s += "    case " + str(i) + ":\n"
            s += "        result = " 
            s += self.printer.doprint(self.params[i].expression) + ";\n"
            s += "        break;\n"
        s += "     default:\n"
        s += "         break;\n"
        s += "    }\n"
        s += "    return result;\n"
        s += "}\n\n"
        #
        s += "static int " + self.module 
        s += "_set_param_value(int index, double value) {\n"
        s += "    int result = 1;\n"
        s += "    switch (index) {\n"
        for i in range(0, len(self.params)):
            s += "    case " + str(i) + ":\n"
            s += "        " + self.printer.doprint(self.params[i].expression) 
            s += " = value;\n"
            s += "        break;\n"
        s += "     default:\n"
        s += "         break;\n"
        s += "    }\n"
        s += "    return result;\n"
        s += "}\n\n"
        #
        self.calib_h = s

    def parse_formula(self, formula_string="Si(1)O(2)"):
        """
        Parses a chemical formula, and returns a molecular weight and an array of
        elemental concentrations.

        Parameters
        ----------
        formula_string : str 
            Formula of compound specified in the standard form
            SiO2 -> Si(1)O(2)

        Returns
        -------
        mw, elmvector : tuple
            - [0] mw is a float containing the molecular weight in grams.
                    
            - [1] elmvector is a numpy array of length 120 containing the mole numbers 
              of each element in the compound.
        """
        formula = formula_string.title().replace('(',',').replace(')',',').split(',')
        mw = 0.0
        elmvector = np.zeros(120)
        for i in range(0,len(formula)-1,2):
            element = self._elements.loc[self._elements['Abbrv'] == formula[i]]
            ind = element.index.values[0]
            mw += element.MW.values[0]*float(formula[i+1])
            elmvector[ind] = float(formula[i+1])
        return mw, elmvector
    
    def get_berman_std_state_database(self, identifier=None, extend_defs=False):
        """
        Retrieves priors from the thermodynamic database of Berman (1988).

        Parameters
        ----------
        identifier : int
            Value may be None or an integer in the range (0, length of Berman 
            database).
        extend_defs : bool
            False: The standard set of Berman parameters are retrieved.  
            
            True: Additional parameters are computed, including 'K' and 
            'K_P', the bulk modulus and its pressure derivative.

        Returns
        -------
        result : variable
            If identifier is None, then the function returns an array of tuples, 
            one for each phase in the database:
            
            - [0] database index  
            
            - [1] name of the phase  
            
            - [2] formula of the phase, in standard notation  
            
            If identifier is an integer, then the function returns a dictionary 
            with keys, corresponding parameter names, and values corresponding 
            to parameter values.
        
        """
        if self._berman_db is None:
            params = ['H_TrPr', 'S_TrPr', 'k0', 'k1', 'k2', 'k3', 'V_TrPr', 
            'v1', 'v2', 'v3', 'v4', 'l1', 'l2', 'k_lambda', 'T_lambda_Pr', 
            'T_lambda_ref', 'H_t', 'd0', 'd1', 'd2', 'd3', 'd4', 'd5', 'T_D', 
            'T_D_ref', 'T_r', 'P_r']
            self._berman_db = pd.read_json(path.join(path.dirname(__file__), 
                DATADIR, 'berman_1988.json'))
            self._berman_db['T_r'] = 298.15 
            self._berman_db['P_r'] = 1.0
            if extend_defs:
                params.append('K')
                self._berman_db['K'] = self._berman_db.apply(
                    lambda row: -1.0/row['v1'] if row['v1'] != 0 else 1000000.0, 
                    axis=1)
                params.append('K_P')
                self._berman_db['K_P'] = self._berman_db.apply(
                    lambda row: (2.0*row['v2']/row['v1']/row['v1']-1.0) \
                    if row['v1'] != 0 and row['v2'] != 0 else 4.0, axis=1)
            self._berman_db = self._berman_db[['Phase', 'Formula'] + params]
            self._berman_db.fillna(0, inplace=True)
        if identifier is None:
            result = []
            for i in range (0, len(self._berman_db)):
                row = self._berman_db.iloc[i,:]
                result.append((i, row.Phase.title(), 
                    row.Formula.title().replace("(1)","").replace("(", "").replace(")","")))
        elif identifier >= 0 and identifier < len(self._berman_db):
            row = self._berman_db.to_dict('records')[identifier]
            result = row
        else:
            result = None
        return result

    def create_code_module(self, phase="Quartz", formula="Si(1)O(2)", params={}, 
        identifier=None, prefix="cy", module_type="fast", silent=False, 
        language='C'):
        """
        Creates and writes an include and code file for a model instance.

        Parameters
        ----------
        phase : str
            Model instance title (e.g., phase name).  Used to name the generated 
            function. Cannot contain blank spaces or special characters; underscore 
            ("_") is permitted. Convention capitalizes the first letter and the 
            letter following an underscore ("_") character.
        formula : str
            Chemical formula of the model instance, in standard notation.
            Standard notation is of the form: Element Symbol followed by 
            parentheses enclosing a number, which may be decimalized.  E.g., 
            SiO2 is written as Si(1)O(2); CaMg1/2Ti1/2AlO6 is written as 
            Ca(1)Mg(0.5)Ti(0.5)Al(1)Si(1)O(6); CaMg(CO3)2 is written as 
            Ca(1)Mg(1)C(2)O(6).
        params : dict
            Parameter values for the model instance.
            The keys of this dictionary are validated against parameter symbols 
            stored for the model.
        identifier : str
            A unique identifier for the model instance.
            Defaults to local date and time when module is created (rounded to 
            the second).
        prefix : str
            Prefix to function names for Python bindings, e.g., 
            {prefix}_{phase}_{module}_g(T,P).
        module_type : str
            Generate code that executes "fast" but does not expose hooks for 
            model parameter calibration. Alternately, generate code suitable for 
            "calib"ration of parameters in the model, which executes more slowly 
            and exposes additional functions that allow setting of parameters 
            and generation of derivatives of thermodynamic functions with 
            respect to model parameters. 
        silent : bool
            Print (True) or do not print (False) status messages.
        language : string
            Language syntax for generated code. ("C" is the C99 programming 
            language; "C++" is the C++ programming language.)

        Returns
        -------
        result : Boolean
                 True if module is succesfully generated, False if some error occurred 
        """
        assert language == "C"
        if not re.match("^[a-zA-Z0-9_]*$", phase):
            print ("Error: ", phase, 
                " is only allowed to have characters a-z, A-Z, 0-9, and _")
            return False
        okay = True
        for param in self.params:
            if not param.name in params:
                print ("Error: paramter key ", param.name, 
                    " is missing from passed model parameter dictionary")
                okay = False
        if not okay:
            print ("Error in params dictionary (see above)")
            return False
        (mw, elmvector) = self.parse_formula(formula)
        if mw == 0.0:
            print ("Error: parsing of the formula ", formula, 
                " retunred a molecular weight of zero")
            return False
        module = self.get_module_name()
        if module == 'untitled':
            print ("Error: Please set module name before called this function")
            return False

        if self.calc_h is None:
            if not silent:
                print ("Creating (once only) generic fast model code file string")
            self.create_calc_h_file(language, module_type)

        if module_type == 'fast':
            if self.fast_h_template is None:
                if not silent:
                    print ("Creating (once only) generic model fast code template include file string")
                self.fast_h_template = tpl.create_fast_h_template()
            if self.fast_c_template is None:
                if not silent:
                    print ("Creating (once only) generic model fast code template code file string")
                self.fast_c_template = tpl.create_fast_c_template()
        elif module_type == 'calib':
            if self.calib_h_template is None:
                if not silent:
                    print ("Creating (once only) generic model calib code template include file string")
                self.calib_h_template = tpl.create_calib_h_template()
            if self.calib_c_template is None:
                if not silent:
                    print ("Creating (once only) generic model calib code template code file string")
                self.calib_c_template = tpl.create_calib_c_template()
            if self.calib_h is None:
                if not silent:
                    print ("Creating (once only) generic calib model code file string")
                self.create_calib_h_file()
        else:
            print ("Error: module_type must be set to either 'fast' or 'calib'")
            return False


        if not silent:
            print ("Creating include file ...")
        h_file = ""
        h_file_name = ""
        if module_type == 'fast':
            h_file = self.fast_h_template.format(module=module, phase=phase)
            h_file_name = phase + "_" + module + "_calc.h"
        elif module_type == 'calib':
            h_file = self.calib_h_template.format(module=module, phase=phase)
            h_file_name = phase + "_" + module + "_calib.h"
        if not silent:
            print ("... done!")

        if not silent:
            print ("Creating code file ...")
        if identifier is None:
            identifier = time.asctime(time.localtime(time.time()))
        formula_condensed = formula.title().replace("(1)","").replace("(", "").replace(")","") 
        param_block = "\n"
        for param in self.params:
            if module_type == 'fast':
                param_block += "static const double " + param.name 
                param_block += " = " + str(params[param.name]) + ";\n"
            elif module_type == 'calib':
                param_block += "static double " + param.name 
                param_block += " = " + str(params[param.name]) + ";\n"
        (param_names, param_values) = zip(*params.items())
        c_file = ""
        c_file_name = ""
        if module_type == 'fast':
            c_file = self.fast_c_template.format( module=module, phase=phase, 
                formula=formula_condensed, mw=mw, elmvector=elmvector, 
                parameter_init_block=param_block, git_identifier=identifier,
                include_calc_h=self.calc_h)
            c_file_name = phase + "_" + module + "_calc.c"
        elif module_type == 'calib':
            c_file = self.calib_c_template.format( module=module, phase=phase, 
                formula=formula_condensed, mw=mw, elmvector=elmvector, 
                parameter_init_block=param_block, git_identifier=identifier,
                include_calc_h=self.calc_h, include_calib_h=self.calib_h)
            c_file_name = phase + "_" + module + "_calib.c"
        if not silent:
            print ("... done")

        if not silent:
            print ("Writing include file to working directory ...")
        with open(h_file_name, 'w') as f:
            f.write(h_file)

        if not silent:
            print ("Writing code file to working directory ...")
        with open(c_file_name, 'w') as f:
            f.write(c_file)

        if not silent:
            print ("Writing pyxbld file to working directory ...")
        pyxbld_template = tpl.create_pyxbld_template()
        pyxbld_file = pyxbld_template.format(file_to_compile=c_file_name)
        pyxbld_file_name = module + ".pyxbld"
        with open(pyxbld_file_name, 'w') as f:
            f.write(pyxbld_file)

        if not silent:
            print ("writing pyx file to working directory ...")
        pyx_template = ""
        if module_type == "fast":
            pyx_template = tpl.create_fast_pyx_template()
        elif module_type == "calib":
            pyx_template = tpl.create_calib_pyx_template()
        pyx_file = pyx_template.format(prefix=prefix, phase=phase, module=module)
        pyx_file_name = module + ".pyx"
        with open(pyx_file_name, 'w') as f:
            f.write(pyx_file)

        # pyximport.install(pyximport=True, pyimport=False, build_dir=None, 
        # build_in_temp=True, setup_args=None, reload_support=False, 
        # load_py_module_on_import_failure=False, inplace=False, 
        # language_level=None)

        if not silent:
            print ("Compiling code and Python bindings ...")
        pyximport.install(language_level=3)

        if not silent:
            print ("Success! Import the module named ", module)
        return True


# Import DEW 2019 Aqueous Species

**To do:**
- Change the code to use a more obvious folder name than 'working'

This notebook runs the code required to generate a database from a set of DEW parameters.

Aaron Wolf wrote functions that make this process easier to implement, originally for use in his carbonated mantle demonstration. The following line runs this file, therefore defining the functions written within it.

In [2]:
%run core.ipynb

/Users/simonmatthews/opt/anaconda3/lib/python3.7/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


The HKF equations are developed in the previous notebook and setup there as a coder module. This is a pre-requisite for this notebook, and so this line runs the previous notebook.

In [3]:
%run HKF_Equations.ipynb

## Import and process the DEW database

I have extracted the aqueous species parameters from the DEW "Aqueous Species Options" worksheet. I renamed some of the complexes so that conventions are applied more consistently. I added a column expressing the formulae in the format require by ENKI. For complexes where their parameters are estimated, this estimation must be done by the coder module, I do not import the DEW spreadsheet calculated estimates.

In [4]:
dew = pd.read_csv('dew2019_cleaned.csv')
dew = dew.fillna(0)
dew

,name,EQ_name,Unnamed: 2,symbol,formula,include,G_ref,H_ref,S_ref,V_ref,...,prd_ac,compl,gas,a1,a2,a3,a4,c1,c2,comments
0,"acetate,aq",ACETATE(AQ),11,CH3COO-,C(2)H(3)O(2),1,-88270,-116180.0,20.60,40.499926,...,0,0,0,10.018000,2.809000,6.720000,-2.894100,26.300000,-3.860000,"revised January 26th, 2016; new a1 value from ..."
1,"acetic-acid,aq",ACETIC(AQ),10,CH3COOH,C(2)H(4)O(2),1,-94760,-116100.0,42.70,52.010022,...,0,0,0,11.500000,5.500000,1.670000,-2.870000,44.900000,-2.630000,Plyasunov & Shock (2001)
2,Ag+,AG+,3,Ag(+),Ag(1),1,18427,25275.0,17.54,-0.800078,...,1,0,0,2.046420,-4.387714,7.018169,-2.597612,12.786274,-1.425436,Shock & Helgeson (1988) revised with new predi...
3,"AgCl,aq",AGCL(AQ),8,AgCl(0),Ag(1)Cl(1),0,-17450,-18270.0,34.10,27.000000,...,1,1,0,6.745843,2.139576,3.044087,-2.867450,9.743185,-1.669791,Sverjensky et al. (1997) with new V greater th...
4,AgCl2-,AGCL2-,6,AgCl2(-),Ag(1)Cl(2),0,-51560,-61130.0,47.00,53.000000,...,1,1,0,12.253325,7.383245,-1.352680,-3.084223,19.126467,-1.466131,Sverjensky et al. (1997) with new V greater th...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
223,Yb+3,YB+3,4,Yb(+3),Yb(1),0,-153000,-160300.0,-56.90,-44.499994,...,1,0,0,-9.223325,-1.432516,14.532520,-2.719780,7.353108,-10.449205,Shock & Helgeson (1988) revised with new predi...
224,Zn+2,ZN+2,4,Zn(+2),Zn(1),1,-35200,-36660.0,-26.20,-24.300094,...,1,0,0,-1.417424,-9.478710,9.977699,-2.387150,15.901037,-4.317967,Shock & Helgeson (1988) revised with new predi...
225,ZnCl+,ZNCL+,5,ZnCl(+),Zn(1)Cl(1),0,-66850,-66240.0,23.00,4.800000,...,1,1,0,2.549738,-1.855533,6.393947,-2.702292,19.694777,1.019011,Sverjensky et al. (1997) with revised a1 equal...
226,"ZnCl2,aq",ZNCL2(AQ),9,ZnCl2(0),Zn(1)Cl(2),0,-98300,-109080.0,27.03,26.000000,...,1,1,0,6.551633,1.954668,3.199130,-2.859806,26.129473,4.025663,Sverjensky et al. (1997) with revised a1 equal...


The parameters given in the DEW spreadsheet are multiplied by factors of 10 for easier display, and the units of energy are colories not Joules. This can be fixed here:

In [5]:
multipliers = {'a1': 4.184/10,
               'a2': 4.184*100,
               'a3': 4.184,
               'a4': 4.184*1e4,
               'c1': 4.184,
               'c2': 4.184*1e4,
               'G_ref': 4.184,
               'H_ref': 4.184,
               'S_ref': 4.184,
               'V_ref': 0.1,
               'Cp_ref': 4.184,
               'omega0': 4.184*1e5}

for col in list(multipliers.keys()):
    dew[col] = dew[col] * multipliers[col]

dew

,name,EQ_name,Unnamed: 2,symbol,formula,include,G_ref,H_ref,S_ref,V_ref,...,prd_ac,compl,gas,a1,a2,a3,a4,c1,c2,comments
0,"acetate,aq",ACETATE(AQ),11,CH3COO-,C(2)H(3)O(2),1,-369321.680,-486097.12,86.19040,4.049993,...,0,0,0,4.191531,1175.285600,28.116480,-121089.144000,110.039200,-161502.400000,"revised January 26th, 2016; new a1 value from ..."
1,"acetic-acid,aq",ACETIC(AQ),10,CH3COOH,C(2)H(4)O(2),1,-396475.840,-485762.40,178.65680,5.201002,...,0,0,0,4.811600,2301.200000,6.987280,-120080.800000,187.861600,-110039.200000,Plyasunov & Shock (2001)
2,Ag+,AG+,3,Ag(+),Ag(1),1,77098.568,105750.60,73.38736,-0.080008,...,1,0,0,0.856222,-1835.819346,29.364018,-108684.082816,53.497768,-59640.249478,Shock & Helgeson (1988) revised with new predi...
3,"AgCl,aq",AGCL(AQ),8,AgCl(0),Ag(1)Cl(1),0,-73010.800,-76441.68,142.67440,2.700000,...,1,1,0,2.822461,895.198426,12.736460,-119974.110301,40.765485,-69864.051884,Sverjensky et al. (1997) with new V greater th...
4,AgCl2-,AGCL2-,6,AgCl2(-),Ag(1)Cl(2),0,-215727.040,-255767.92,196.64800,5.300000,...,1,1,0,5.126791,3089.149764,-5.659614,-129043.905131,80.025136,-61342.929492,Sverjensky et al. (1997) with new V greater th...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
223,Yb+3,YB+3,4,Yb(+3),Yb(1),0,-640152.000,-670695.20,-238.06960,-4.449999,...,1,0,0,-3.859039,-599.364582,60.804063,-113795.586832,30.765404,-437194.758120,Shock & Helgeson (1988) revised with new predi...
224,Zn+2,ZN+2,4,Zn(+2),Zn(1),1,-147276.800,-153385.44,-109.62080,-2.430009,...,1,0,0,-0.593050,-3965.892286,41.746694,-99878.361272,66.529939,-180663.755263,Shock & Helgeson (1988) revised with new predi...
225,ZnCl+,ZNCL+,5,ZnCl(+),Zn(1)Cl(1),0,-279700.400,-277148.16,96.23200,0.480000,...,1,1,0,1.066810,-776.354860,26.752275,-113063.908995,82.402946,42635.437729,Sverjensky et al. (1997) with revised a1 equal...
226,"ZnCl2,aq",ZNCL2(AQ),9,ZnCl2(0),Zn(1)Cl(2),0,-411287.200,-456390.72,113.09352,2.600000,...,1,1,0,2.741203,817.833260,13.385159,-119654.282705,109.325714,168433.725945,Sverjensky et al. (1997) with revised a1 equal...


Create a function to generate the parameter dictionary required for each phase by the coder module. Create a name for the species that can be used in filenames.

In [6]:
def HKF_params(species='H+',Formula='', z=0.0,
                   G_ref=0.0, S_ref=0.0, V_ref=0.0, Cp_ref = 0.0,
                   a1=0.0, a2=0.0, a3=0.0, a4=0.0,
                   c1=0.0, c2=0.0, omega0=0.0,
                   theta=228.0, Psi=2600.0, eta=694657.0, rH=3.082,
                   T_r=298.15, P_r=1.0):
    
    species_name = species
    
    if species_name[-1] == '-':
        species_name = species_name[:-1]+'_n'
    if species_name[-2] == '-':
        species_name = species_name[:-2]+'_n'+species_name[-1]
    species_name = species_name.replace('-','_')
    species_name = species_name.replace('+','_p')
    species_name = species_name.replace('(','_l_')
    species_name = species_name.replace(')','_r_')
        
    species_name = species_name.replace(',','_')
    
    param_dict = {'Phase': species_name,
                  'Formula': Formula,
                  'G_ref':G_ref,
                  'S_ref':S_ref,
                  'v_ref':V_ref,
                  'C_p_ref':Cp_ref,
                  'a1':a1,
                  'a2':a2,
                  'a3':a3,
                  'a4':a4,
                  'c1':c1,
                  'c2':c2,
                  'omega0':omega0,
                  'theta':theta,
                  'Psi':Psi,
                  'eta':eta,
                  'rH':rH,
                  'z':z,
                  'T_r':T_r,
                  'P_r':P_r}
    return param_dict

## Create objects for each DEW species
First, create a dictionary of phase parameters for each species

**NOTE: To use the 'prettier' complex names that are not entirely capitalised, change row['EQ_name'] to row['name']**. Capitalised names ensure compatibility with all versions of EQ3 and EQ6.

In [7]:
phase_params = {}
for i, row in dew.iterrows():
    phase_params[row['EQ_name']] = HKF_params(species=row['EQ_name'],Formula=row['formula'], z=row['z'],
                   G_ref=row['G_ref'], S_ref=row['S_ref'], V_ref=row['V_ref'], Cp_ref = row['Cp_ref'],
                   a1=row['a1'], a2=row['a2'], a3=row['a3'], a4=row['a4'],
                   c1=row['c1'], c2=row['c2'], omega0=row['omega0'])

Build the database, including generating the coder files

In [8]:
mod_name = 'dew2019'
modelDB, codermod = make_custom_database(mod_name,phase_params,working_dir='dew2019_coderfiles')

/Users/simonmatthews/repos/pyQ3/dew2019_coderfiles
Creating (once only) generic fast model code file string
Creating (once only) generic model fast code template include file string
Creating (once only) generic model fast code template code file string
Creating include file ...
... done!
Creating code file ...
... done
Writing include file to working directory ...
Writing code file to working directory ...
Writing pyxbld file to working directory ...
writing pyx file to working directory ...
Compiling code and Python bindings ...
Success! Import the module named  dew2019
Creating include file ...
... done!
Creating code file ...
... done
Writing include file to working directory ...
Writing code file to working directory ...
Writing pyxbld file to working directory ...
writing pyx file to working directory ...
Compiling code and Python bindings ...
Success! Import the module named  dew2019
Creating include file ...
... done!
Creating code file ...
... done
Writing include file to worki

/Users/simonmatthews/opt/anaconda3/lib/python3.7/site-packages/Cython/Compiler/Main.py:369: FutureWarning: Cython directive 'language_level' not set, using 2 for now (Py2). This will change in a later release! File: /Users/simonmatthews/repos/pyQ3/dew2019_coderfiles/dew2019.pyx
  tree = Parsing.p_module(s, pxd, full_module_name)


/Users/simonmatthews/repos/pyQ3


## Check database
I'm not sure putting all the species into a database is the right thing to do since they're not individual phases. Consider this later once the necessary functionality has been identified.

In [9]:
modelDB.phase_info

,abbrev,phase_name,formula,phase_type,endmember_num
0,ACETATE(AQ),ACETATE_l_AQ_r_,C2H3O2,pure,1
1,ACETIC(AQ),ACETIC_l_AQ_r_,C2H4O2,pure,1
2,AG+,AG_p,Ag,pure,1
3,AGCL(AQ),AGCL_l_AQ_r_,AgCl,pure,1
4,AGCL2-,AGCL2_n,AgCl2,pure,1
...,...,...,...,...,...
223,YB+3,YB_p3,Yb,pure,1
224,ZN+2,ZN_p2,Zn,pure,1
225,ZNCL+,ZNCL_p,ZnCl,pure,1
226,ZNCL2(AQ),ZNCL2_l_AQ_r_,ZnCl2,pure,1


## Save output phases

In [10]:
output = {}
for phsnm in phase_params:
    iphs = modelDB.get_phase(phsnm)
    output[phsnm] = iphs
output

{'ACETATE(AQ)': <thermoengine.phases.PurePhase at 0x7fe4c35c6a50>,
 'ACETIC(AQ)': <thermoengine.phases.PurePhase at 0x7fe4c35cc650>,
 'AG+': <thermoengine.phases.PurePhase at 0x7fe4c35c6a90>,
 'AGCL(AQ)': <thermoengine.phases.PurePhase at 0x7fe4c35c1390>,
 'AGCL2-': <thermoengine.phases.PurePhase at 0x7fe4c35c1fd0>,
 'AL+3': <thermoengine.phases.PurePhase at 0x7fe4d4cee810>,
 'ALO2-': <thermoengine.phases.PurePhase at 0x7fe4c35c1690>,
 'ALO2(SIO2)-': <thermoengine.phases.PurePhase at 0x7fe4d45a2910>,
 'AR(AQ)': <thermoengine.phases.PurePhase at 0x7fe4c35c1590>,
 'AU+': <thermoengine.phases.PurePhase at 0x7fe4c35c1a10>,
 'AU+3': <thermoengine.phases.PurePhase at 0x7fe4c35c1790>,
 'B(OH)3(AQ)': <thermoengine.phases.PurePhase at 0x7fe4d4b203d0>,
 'BA+2': <thermoengine.phases.PurePhase at 0x7fe4c35c1850>,
 'BACL+': <thermoengine.phases.PurePhase at 0x7fe4c35c1d50>,
 'BE+2': <thermoengine.phases.PurePhase at 0x7fe4c35c1d10>,
 'BENZENE(AQ)': <thermoengine.phases.PurePhase at 0x7fe4c35c1ed0>,

In [11]:
with open('DEW2019.pkl','wb') as file:
    dill.dump(output, file)
    file.close()